# Python HANA ML APL

<div class="alert alert-block alert-info">
<b>Reports on cashflow time series model.</b> <br>
</div>

### Create an HANA Dataframe for the actual series

In [1]:
from hana_ml import dataframe as hd
conn = hd.ConnectionContext(userkey='MLMDA_KEY')
sql_cmd = 'SELECT "Date", "Cash", "MondayMonthInd", "FridayMonthInd" FROM "APL_SAMPLES"."CASHFLOWS_FULL" ORDER BY 1'
series_in = hd.DataFrame(conn, sql_cmd)

In [2]:
series_in.head(5).collect()

,Date,Cash,MondayMonthInd,FridayMonthInd
0,2001-01-02,4586.144145,0,0
1,2001-01-03,3750.2201,0,0
2,2001-01-04,4978.993643,0,0
3,2001-01-05,5803.562791,0,1
4,2001-01-08,5711.32164,1,0


### Fit with APL

#### Modeling

In [3]:
from hana_ml.algorithms.apl.time_series import AutoTimeSeries
apl_model = AutoTimeSeries(time_column_name= 'Date', target= 'Cash', horizon= 21)
apl_model.set_params(
    other_params = {'force_positive_forecast': True},
    other_train_apl_aliases = {'APL/DecomposeInfluencers':'true'})
apl_model.fit(data = series_in)

 #### Reports

In [4]:
df = apl_model.get_debrief_report('TimeSeries_ModelOverview').collect()
df = df[['Target Variable','First Date','Last Date','Horizon','Granularity']]
df.style.hide_index()

Target Variable,First Date,Last Date,Horizon,Granularity
Cash,2001-01-02,2001-12-28,21,dayAmplitude


In [5]:
my_filter = "\"Partition\"='Validation'"
df = apl_model.get_debrief_report('TimeSeries_Performance').filter(my_filter).collect()
df = df[['Partition','MAE','MAPE','RMSE']]
format_dict = {'MAE':'{:,.3f}','MAPE':'{:,.1f}%','RMSE':'{:,.3f}'}
df.style.format(format_dict).hide_index()

Partition,MAE,MAPE,RMSE
Validation,936.118,14.3%,"1,301.372"


In [6]:
df = apl_model.get_debrief_report('TimeSeries_Components').collect()
df = df[['Component Type','Component Value']]
df.style.hide_index()

Component Type,Component Value
Trend,Polynom( Date)
Cycles,Sum(PeriodicExtrasPred_MondayMonthInd)
Fluctuations,None


In [7]:
df = apl_model.get_debrief_report('TimeSeries_Decomposition').collect()
df = df[['Row','Type','Item','Relative Impact']]
df = df.sort_values(by=['Row'])
format_dict = {'Relative Impact':'{:,.3f}'}
df.style.format(format_dict).hide_index()

Row,Type,Item,Relative Impact
1,Trend,Quadratic,0.840
2,Cycles,None,0.000
3,Influencers,MondayMonthInd,0.075
4,Fluctuations,None,0.000
5,Residuals,None,0.085


In [8]:
df = apl_model.get_debrief_report('TimeSeries_Outliers').collect()
df = df[['Partition','Date','Signal','Forecast']]
df.style.hide_index()

Partition,Date,Signal,Forecast
Estimation,2001-02-16,8888.033000,4376.910000
Estimation,2001-02-19,24659.179000,19248.851000
Estimation,2001-04-24,8096.141000,3965.624000
Estimation,2001-08-20,14552.070000,19446.136000
Validation,2001-12-14,11464.127000,6733.448000


### Predict with APL

In [9]:
apl_model.set_params(
    extra_applyout_settings = {'APL/ApplyExtraMode': 'First Forecast with Stable Components and Residues and Error Bars'})
series_out = apl_model.predict(data = series_in)
df_out = series_out.collect()

In [10]:
import pandas as pd
with pd.option_context('display.float_format', '{:,.1f}'.format):
    display(df_out.tail(7))

,Date,ACTUAL,LOWER_INT_95PCT,UPPER_INT_95PCT,PREDICTED_1,Trend_1,Cycles_1,ExtraPreds_1,Fluctuations_1,Residues_1
265,2002-01-23,None,"5,310.1","10,411.5","7,860.8","7,797.4",0.0,63.4,0.0,nan
266,2002-01-24,None,"5,340.7","10,442.1","7,891.4","7,828.0",0.0,63.4,0.0,nan
267,2002-01-25,None,"5,371.5","10,472.9","7,922.2","7,858.8",0.0,63.4,0.0,nan
268,2002-01-28,None,"4,885.4","9,986.7","7,436.0","7,951.8",0.0,-515.7,0.0,nan
269,2002-01-29,None,"5,495.7","10,597.1","8,046.4","7,983.0",0.0,63.4,0.0,nan
270,2002-01-30,None,"5,527.0","10,628.4","8,077.7","8,014.3",0.0,63.4,0.0,nan
271,2002-01-31,None,"5,558.5","10,659.9","8,109.2","8,045.8",0.0,63.4,0.0,nan
